In [4]:
import pandas as pd
average_correlation = pd.read_csv('../average_correlation_matrix.csv', index_col=0)
average_correlation

,K1,K14,K15,K2,K3,K4,K5,K7,LANA,Orf10,...,Orf67,Orf68,Orf69,Orf7,Orf70,Orf8,Orf9,vIRF1,vIRF2,vIRF4
K1,1.000000,0.431397,0.089735,0.271017,0.184781,0.290918,0.373761,0.313811,0.340167,0.370072,...,0.248528,-0.018085,0.251307,0.396904,0.238422,0.398816,0.470258,0.229243,0.211747,0.223686
K14,0.431397,1.000000,0.557989,0.594787,0.520712,0.507301,0.443679,0.609572,0.769794,0.611961,...,0.627707,0.360511,0.715968,0.634914,0.455124,0.626742,0.599966,0.751996,0.528935,0.687882
K15,0.089735,0.557989,1.000000,0.398219,0.144964,0.112921,0.009892,0.338204,0.740032,0.333666,...,0.432647,0.715067,0.829518,0.359940,0.156643,0.313608,0.208144,0.639593,0.472855,0.527738
K2,0.271017,0.594787,0.398219,1.000000,0.580200,0.634742,0.613633,0.771647,0.717503,0.840891,...,0.667291,0.190407,0.708208,0.708406,0.594477,0.841821,0.738512,0.591926,0.367530,0.741872
K3,0.184781,0.520712,0.144964,0.580200,1.000000,0.837607,0.593356,0.657184,0.413023,0.684201,...,0.596828,-0.019912,0.375666,0.492431,0.833162,0.545447,0.633381,0.508109,0.293566,0.655102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Orf8,0.398816,0.626742,0.313608,0.841821,0.545447,0.602038,0.574024,0.711349,0.626339,0.850759,...,0.712416,0.136123,0.636263,0.767337,0.472981,1.000000,0.871275,0.586737,0.287060,0.765628
Orf9,0.470258,0.599966,0.208144,0.738512,0.633381,0.650009,0.624743,0.750595,0.559953,0.835250,...,0.710768,0.027649,0.561446,0.775450,0.577688,0.871275,1.000000,0.483436,0.202731,0.750208
vIRF1,0.229243,0.751996,0.639593,0.591926,0.508109,0.479244,0.257162,0.497789,0.697491,0.574568,...,0.741132,0.568322,0.755250,0.582306,0.395890,0.586737,0.483436,1.000000,0.661492,0.852661
vIRF2,0.211747,0.528935,0.472855,0.367530,0.293566,0.353589,0.068140,0.335008,0.487966,0.307617,...,0.291429,0.584525,0.476167,0.408721,0.285215,0.287060,0.202731,0.661492,1.000000,0.453974
